In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from arch.univariate import GARCH, ConstantMean, Normal

sns.set_style("darkgrid")
plt.rc("figure", figsize=(16, 6))
plt.rc("savefig", dpi=90)
plt.rc("font", family="sans-serif")
plt.rc("font", size=14)


import arch.data.nasdaq

data = arch.data.nasdaq.load()
nasdaq = data["Adj Close"]
print(nasdaq.head())

Date
1999-01-04    2208.050049
1999-01-05    2251.270020
1999-01-06    2320.860107
1999-01-07    2326.090088
1999-01-08    2344.409912
Name: Adj Close, dtype: float64


In [3]:
rets = 100 * nasdaq.pct_change().dropna()

# Build components to set the state for the distribution
random_state = np.random.RandomState(1)
dist = Normal(random_state=random_state)
volatility = GARCH(1, 1, 1)

mod = ConstantMean(rets, volatility=volatility, distribution=dist)

C:\dev\projects\menashe\venv\lib\site-packages\arch\univariate\distribution.py:63: FutureWarning: random_state is deprecated and will be removed in a future update. Use seed instead.
  warnings.warn(


In [4]:
res = mod.fit(disp="off")
res

                   Constant Mean - GJR-GARCH Model Results                    
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                  GJR-GARCH   Log-Likelihood:               -8196.75
Distribution:                  Normal   AIC:                           16403.5
Method:            Maximum Likelihood   BIC:                           16436.1
                                        No. Observations:                 5030
Date:                Fri, May 05 2023   Df Residuals:                     5029
Time:                        15:13:35   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0376  1.476e-02      2.549  1.081e-02 [8.

In [47]:
forecasts = res.forecast(
    start=rets.index[-1], 
    horizon=10, 
    reindex=False, 
    simulations=10, 
    method='simulation',
    # method='analytic',
    # method='bootstrap',
)
print(forecasts.residual_variance.dropna().head())

                h.01      h.02      h.03      h.04      h.05      h.06  \
Date                                                                     
2018-12-31  4.264752  4.103844  3.984441  4.116726  3.908938  3.753886   

                h.07      h.08      h.09      h.10  
Date                                                
2018-12-31  3.507736  3.662643  4.122299  4.606158  


In [51]:
forecasts.simulations.residual_variances.mean(axis=1)

array([[4.26475215, 4.103844  , 3.98444086, 4.11672647, 3.90893787,
        3.75388572, 3.50773583, 3.6626425 , 4.12229926, 4.60615773]])